So far we've learnt how to scrape the web, and how to make a request for information from an API. Some websites make APIs even easier. Check out [RapidAPI](https://rapidapi.com/) they take care of writing most of the code for you.

We will use the [AeroDataBox API](https://rapidapi.com/aedbx-aedbx/api/aerodatabox/), which can retrieve all sorts of information about flights and airports. We will show you how to retrieve information about the airports, and then it's up to you to apply this, along with what you've already learnt this week, to **produce a function, which retrieves tomorrows flight information for the major airports in the cities you web scraped**.

In [1]:
import pandas as pd
import requests

On the left hand side of the AeroDataBox API page, you'll see a list of options for information that you can retrieve:
> - Flights API
- Subsciption / PUSH API
- Airport API
- Aircraft API
- Healthcheck & Status API

1. We want to select `Airport API`

2. Then within Airport API we want to select `Search airports by location`

3. Now in the middle third you'll want to enter the `latitude` and `longitude` of any city to test... we chose Berlin: latitude 52.31 longitude 13.24. Next we changed the `radiusKM` to only 50km. And finally set `withFlightInfoOnly` to true, so it will only return airports which have flight data (scheduled or live) available.

4. On the right hand third of the screen you should see a block of code that looks pretty unfamiliar. This is because by default the code is probably set to *(Node.js) Axios*. However, we have the power to change this to familiar python. Select the dropdown box at the top of the code and select `python > requests`.

Now you can copy the code to your notebook and it should look a little something like the cell below:

In [2]:
import requests

url = "https://aerodatabox.p.rapidapi.com/airports/search/location/52.31/13.24/km/50/10"

querystring = {"withFlightInfoOnly":"true"}

headers = {
	"X-RapidAPI-Key": "14a44098c8mshe4536a007985112p1e3b4bjsn8fd805eb6bd4", #"YOUR_API_KEY_HERE",
	"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

{"searchBy":{"lat":52.31,"lon":13.24},"count":1,"items":[{"icao":"EDDB","iata":"BER","name":"Berlin Brandenburg","shortName":"Brandenburg","municipalityName":"Berlin","location":{"lat":52.35139,"lon":13.493889},"countryCode":"DE"}]}


Let's view the response as `.json()` instead of `.text` so that it's easier to read

In [3]:
response.json()

{'searchBy': {'lat': 52.31, 'lon': 13.24},
 'count': 1,
 'items': [{'icao': 'EDDB',
   'iata': 'BER',
   'name': 'Berlin Brandenburg',
   'shortName': 'Brandenburg',
   'municipalityName': 'Berlin',
   'location': {'lat': 52.35139, 'lon': 13.493889},
   'countryCode': 'DE'}]}

We can now turn this into a dataframe using `.json_normalize()`

In [4]:
pd.json_normalize(response.json()['items'])

,icao,iata,name,shortName,municipalityName,countryCode,location.lat,location.lon
0,EDDB,BER,Berlin Brandenburg,Brandenburg,Berlin,DE,52.35139,13.493889


Let's now use this for the latitude and longitude of multiple cities

In [6]:
def icao_airport_codes(latitudes, longitudes):

  #assert len(latitudes) == len(longitudes)

  list_for_df = []

  for index, value in enumerate(latitudes):

    url = f"https://aerodatabox.p.rapidapi.com/airports/search/location/{value}/{longitudes[index]}/km/50/10"

    querystring = {"withFlightInfoOnly":"true"}

    headers = {
      "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com",
      "X-RapidAPI-Key": "14a44098c8mshe4536a007985112p1e3b4bjsn8fd805eb6bd4"
    }

    response = requests.request("GET", url, headers=headers, params=querystring)

    list_for_df.append(pd.json_normalize(response.json()['items']))

  return pd.concat(list_for_df, ignore_index=True)

In [8]:
icao_airport_codes()

TypeError: icao_airport_codes() missing 2 required positional arguments: 'latitudes' and 'longitudes'

In [11]:
latitudes

[52.52, 48.8567, 51.5072]

In [12]:
list_for_df

NameError: name 'list_for_df' is not defined

In [13]:
list(enumerate(latitudes))

[(0, 52.52), (1, 48.8567), (2, 51.5072)]

In [10]:
# coordinates for Berlin, Paris, London
latitudes = [52.5200, 48.8567, 51.5072]
longitudes = [13.4050, 2.3522, -0.1275]

#icao_airport_codes(latitudes, longitudes)

In [24]:
def push_airport_data_to_sql():

  #assert len(latitudes) == len(longitudes)
    
    import pandas as pd
    import requests
    from sqlalchemy import create_engine, inspect

    schema = "gans"
    host = "127.0.0.1"
    user = "root"
    password = "MySQLjau"
    port = 3306
    connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

    
    latitudes = (pd.read_sql("cities", con=connection_string))["lat"]
    longitudes = (pd.read_sql("cities", con=connection_string))["lon"]

    list_for_df = []

    for index, value in enumerate(latitudes):

        url = f"https://aerodatabox.p.rapidapi.com/airports/search/location/{value}/{longitudes[index]}/km/50/10"

        querystring = {"withFlightInfoOnly":"true"}

        headers = {
        "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com",
        "X-RapidAPI-Key": "14a44098c8mshe4536a007985112p1e3b4bjsn8fd805eb6bd4"
        }

        response = requests.request("GET", url, headers=headers, params=querystring)

        list_for_df.append(pd.json_normalize(response.json()['items']))

        airports_df = pd.concat(list_for_df, ignore_index=True)

        engine = create_engine(connection_string)
        inspector = inspect(engine)
        if 'airports' in inspector.get_table_names():
          existing_data = pd.read_sql('airports', engine)
          airports_df = airports_df[~airports_df['iata'].isin(existing_data['iata'])]

        airports_df.to_sql('airports',
                  if_exists='append',
                  con=connection_string,
                  index=False)

    return airports_df



In [25]:
push_airport_data_to_sql()

,icao,iata,name,shortName,municipalityName,countryCode,location.lat,location.lon
3,EDDB,BER,Berlin Brandenburg,Brandenburg,Berlin,DE,52.35139,13.493889


###### **Challenge:** Arrivals information
Using what you have been shown above, plus the skills you've learnt in the last couple of days:
1. In `AeroDataBox API` use the `Flight API` > `FIDS/Schedules: Airport departures and arrivals by airport ICAO code` section
2. Fill out the parameters in the middle third and then copy the `python: requests` code from the right hand third
3. Explore the data you get back. What would be useful in your DataFrame and what can be excluded? Remember Gans wants to know about when people are arriving in the city
4. Make a DataFrame from the information you see as important
5. Condense everything you did above into a function that can take a list of ICAO codes as an input, and as an output gives you a DataFrame with the information for *tomorrows arrivals*

In [50]:
def push_cities_airports_to_sql():

    import pandas as pd
    from sqlalchemy import create_engine, inspect
    

    schema = "gans"
    host = "127.0.0.1"
    user = "root"
    password = "MySQLjau"
    port = 3306
    connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

    cities_df = pd.read_sql("cities", con=connection_string)
    airports_df = pd.read_sql("airports", con=connection_string)

    cities_airports_df = (pd.merge(cities_df, airports_df, left_on="city", right_on="municipalityName", how="left"))[["city_id","iata"]]
    cities_airports_df

    engine = create_engine(connection_string)
    inspector = inspect(engine)
    if 'cities_airports' in inspector.get_table_names():
        existing_data = pd.read_sql('cities_airports', engine)
        cities_airports_df = cities_airports_df[~cities_airports_df['city_id'].isin(existing_data['city_id'])]

    cities_airports_df.to_sql('cities_airports',
            if_exists='append',
            con=connection_string,
            index=False)
    
    return cities_airports_df

In [52]:
push_cities_airports_to_sql()

,city_id,iata
0,2867714,MUC
1,2911298,HAM
2,2944388,BRE
3,2950159,BER


In [114]:
import requests
from datetime import datetime, timedelta
from sqlalchemy import create_engine, inspect

schema = "gans"
host = "127.0.0.1"
user = "root"
password = "MySQLjau"
port = 3306
connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

iata_list = (pd.read_sql("cities_airports", con=connection_string))["iata"]
current_time = datetime.now()
start_time = current_time.strftime("%Y-%m-%dT%H:%M")
end_time = (current_time + timedelta(hours=12)).strftime("%Y-%m-%dT%H:%M")

querystring = {"withLeg":"false","direction":"Arrival","withCancelled":"false","withCodeshared":"false", "withCargo":"false","withPrivate":"false","withLocation":"false"}

headers = {
	"X-RapidAPI-Key": "14a44098c8mshe4536a007985112p1e3b4bjsn8fd805eb6bd4",
	"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
}

flights_dict = {
	"flight_num": [],
	"departure_iata": [], 
	"arrival_iata": [], 
	"arrival_time": []}

for iata in iata_list:
    
	url = f"https://aerodatabox.p.rapidapi.com/flights/airports/iata/{iata}/{start_time}/{end_time}"
	flight_data_raw = requests.get(url, headers=headers, params=querystring).json()

	for arrival in flight_data_raw["arrivals"]:
		departure_iata = arrival["movement"]["airport"].get("iata", "Unknown")
		arrival_time = arrival["movement"].get("revisedTime", {}).get("local") or arrival["movement"]["scheduledTime"]["local"]
		flights_dict["flight_num"].append(arrival["number"])
		flights_dict["departure_iata"].append(departure_iata)
		flights_dict["arrival_iata"].append(iata)
		flights_dict["arrival_time"].append(datetime.strptime(arrival_time, "%Y-%m-%d %H:%M%z").strftime("%Y-%m-%d %H:%M:%S"))

flights_df = pd.DataFrame(flights_dict)

engine = create_engine(connection_string)
inspector = inspect(engine)

if 'flights' in inspector.get_table_names():
	existing_data = pd.read_sql('flights', engine)
	flights_df = flights_df[~flights_df['flight_num'].isin(existing_data['flight_num'])]

flights_df.to_sql('flights',
				con=engine,
				if_exists='append',
				index=False
				)


152

In [111]:
flights_df.dtypes

flight_num        object
departure_iata    object
arrival_iata      object
arrival_time      object
dtype: object

In [83]:
from datetime import datetime
datetime.strptime(flight_data_raw["arrivals"][0]["movement"]["revisedTime"]["local"], "%Y-%m-%d %H:%M%z").strftime("%Y-%m-%d %H:%M:%S")

'2024-03-11 08:02:00'

In [60]:
flight_data_raw

{'arrivals': [{'movement': {'airport': {'icao': 'EDDL',
     'iata': 'DUS',
     'name': 'Duesseldorf'},
    'scheduledTime': {'utc': '2024-03-11 06:55Z',
     'local': '2024-03-11 07:55+01:00'},
    'revisedTime': {'utc': '2024-03-11 07:02Z',
     'local': '2024-03-11 08:02+01:00'},
    'terminal': '1',
    'gate': 'A02',
    'baggageBelt': 'A1',
    'quality': ['Basic', 'Live']},
   'number': 'EW 9048',
   'callSign': 'GWI52FP',
   'status': 'Arrived',
   'codeshareStatus': 'IsOperator',
   'isCargo': False,
   'aircraft': {'reg': 'D-AGWE', 'modeS': '3C5EE5', 'model': 'Airbus A319'},
   'airline': {'name': 'Eurowings', 'iata': 'EW', 'icao': 'EWG'}},
  {'movement': {'airport': {'icao': 'EFHK', 'iata': 'HEL', 'name': 'Helsinki'},
    'scheduledTime': {'utc': '2024-03-11 07:00Z',
     'local': '2024-03-11 08:00+01:00'},
    'revisedTime': {'utc': '2024-03-11 07:03Z',
     'local': '2024-03-11 08:03+01:00'},
    'runwayTime': {'utc': '2024-03-11 07:09Z',
     'local': '2024-03-11 08:09+0